In [202]:
#Imports 

%matplotlib inline

import matplotlib.pyplot as plt
import time as time
import numpy as np
from scipy import optimize as opt
from numpy.lib import scimath
from pylab import rcParams
rcParams['figure.figsize'] = 7.5, 7.5

import warnings; warnings.simplefilter('ignore')

## Compute Orbital Elements

In [174]:
def orbitalElements(R, V, mu, printResults):

    v = np.linalg.norm(V)

    #Define unit vectors
    x = [1, 0, 0]
    y = [0, 1, 0]
    z = [0, 0, 1]

    #Calculate h
    H = np.cross(R,V)
    h = np.linalg.norm(H)

    #Calculate p
    p = h**2/mu

    #Calculate e
    eVec = np.cross(V, H)/mu - (R/np.linalg.norm(R))
    e = np.linalg.norm(eVec)

    #Calculate a
    a = p/(1-e**2)

    #Calculate i
    i = np.arccos(H[2]/h)

    #Calculate RAAN (big omega)
    n = np.cross(z,H)

    if (np.dot(n,y) >= 0):
        RAAN = np.arccos(n[0]/np.linalg.norm(n))
    elif (np.dot(n,y) < 0):
        RAAN = -np.arccos(n[0]/np.linalg.norm(n))

    #Calculate argument of periapsis (little omega)
    if (np.dot(z, eVec) >= 0):
        aop = np.arccos(np.dot(n,eVec)/(np.linalg.norm(n)*e))
    elif  (np.dot(z,eVec) < 0):
        aop = 2*np.pi - np.arccos(np.dot(n,eVec)/(np.linalg.norm(n)*e))

    #Calculate true anomaly
    if (np.dot(R, V) >= 0):
        theta = np.arccos(np.dot(R,eVec)/(np.linalg.norm(R)*e))
    elif (np.dot(R, V) < 0):
        theta = 2*np.pi - np.arccos(np.dot(R,eVec)/(np.linalg.norm(R)*e))

    if printResults:
        print('Semi major axis a = ', a, ' km')
        print('Eccentricity e = ', e)
        print('Inclination angle i = ', np.degrees(i), ' degrees')
        print('Right ascension of ascending node = ', np.degrees(RAAN), ' degrees')
        print('Argument of periapsis = ', np.degrees(aop), ' degrees')
        print('True anomaly = ', np.degrees(theta), ' degrees')
        
    return([a, e, i, RAAN, aop, theta, h, p])

## Convert Between theta and E

In [175]:
def theta2E(theta, e):
    E = 2*np.arctan(np.sqrt((1-e)/(1+e))*np.tan(theta/2))
    return(E)

In [176]:
def E2theta(E, e):
    theta = 2*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(E/2))
    return(theta)

## Kepler's Equation Minus n*dt

In [177]:
def Kepler(E2, info):
    #print("In Kepler: info = ", info)
    n = info[0]
    e = info[1]
    dt = info[2]*3600 #convert hours to seconds
    E1 = info[3]
    return (E2 - e*np.sin(E2) - (E1 - e*np.sin(E1)) - n*dt)

## Use f and g to Find Rf and Vf

In [178]:
def fgTheta(Ri, Vi, rf, mu, p, deltaTheta):
    
    ri = np.linalg.norm(Ri)
    vi = np.linalg.norm(Vi)
    
    f = 1 - ((rf/p)*(1 - np.cos(deltaTheta)))
    g = (ri*rf/np.sqrt(mu*p))*(np.sin(deltaTheta))

    fdot = np.sqrt(mu/p)*np.tan(deltaTheta/2)*(((1 - np.cos(deltaTheta))/p) - (1/rf) - (1/ri))
    gdot = 1-((ri/p)*(1-np.cos(deltaTheta)))
        
    R2 = np.multiply(f, Ri) + np.multiply(g, Vi)
    V2 = np.multiply(fdot, Ri) + np.multiply(gdot, Vi)
    
    return([R2, V2])

In [179]:
def fgE(Ri, Vi, rf, dt, a, mu, deltaE):
    
    deltat = dt*3600 #hours to seconds
    
    ri = np.linalg.norm(Ri)
    vi = np.linalg.norm(Vi)
    
    f = 1 - ((a/ri)*(1 - np.cos(deltaE)))
    g = deltat - (np.sqrt((a**3)/mu)*(deltaE - np.sin(deltaE)))
    
    fdot = (-np.sin(deltaE)*np.sqrt(mu*a))/(ri*rf)
    gdot = 1 - (1/rf)*(1 - np.cos(deltaE))
    
    R2 = np.multiply(f, Ri) + np.multiply(g, Vi)
    V2 = np.multiply(fdot, Ri) + np.multiply(gdot, Vi)
    
    return([R2, V2])

## Secant Method Solver

In [180]:
def solve(func, info, x0, x1, err, Nmax, isVerbose):
    
    n = 0
    
    while (n < Nmax):
        
        n = n+1
        if isVerbose:
            print("In solve: Iteration: ", n)
            print("In solve: x0 = ", x0)
            print("In solve: x1 = ", x1)
        
        x2 = x1 - func(x1, info)*((x1-x0)/(func(x1, info) - func(x0, info)))
        if isVerbose:
            print("In solve: x2 = ", x2)
        
        if (np.abs(x2 - x1) < err):
            print("\nIn solve: Successful Solve.\n")
            return x2
        
        else:
            x0 = x1
            x1 = x2
            
    return False

## Lambert's Equation Function

In [181]:
def TOFLambert(a, info):
    
    mu = info[0]
    s = info[1]
    c = info[2]
    transferLessThan180 = info[4]
    transferGreaterThan180 = not transferLessThan180
    shortWay = info[5]
    longWay = not shortWay
    isVerbose = info[6]
    
    n = np.sqrt(mu/(a**3))
    
    #Set alpha and beta
    
    if transferLessThan180:
        beta = 2*(np.arcsin(np.sqrt((s - c)/(2*a))))
        if isVerbose:
            print("In TOFLambert, Transfer less than 180")
    elif transferGreaterThan180:
        beta = -2*(np.arcsin(np.sqrt((s - c)/(2*a))))
        if isVerbose:
            print("In TOFLambert, Transfer greater than 180")
        
    if shortWay:
        alpha = 2*np.arcsin(np.sqrt(s/(2*a)))
        if isVerbose:
            print("In TOFLambert, Short way")
    elif longWay:
        alpha = 2*np.pi-2*np.arcsin(np.sqrt(s/(2*a)))
        if isVerbose:
            print("In TOFLambert, Long way")

    TOF = (1/n)*((alpha - beta) - (np.sin(alpha) - np.sin(beta)))/3600
    
    if isVerbose:
        print("In TOFLambert, a = ", a, ' km')
        print("In TOFLambert, alpha = ", alpha)
        print("In TOFLambert, beta = ", beta)
        print("In TOFLambert, n = ", n)
        print("In TOFLambert, s = ", s)
        print("In TOFLambert, c = ", c)
        print("In TOFLambert, TOFi = ", TOF, ' hours')
    
    return TOF

## Lambert's Equation Minus Desired TOF (for root finding)

In [182]:
def TOFLambertSolve(a, info):
    TOFDesired = info[3]
    return (TOFLambert(a, info) - TOFDesired)

## Compute Lambert Transfer Arc

In [183]:
#state1 (list of 2 3-d vectors): initial position and velocity vectors
#state2 (list of 2 3-d vectors): final position and velocity vectors
#transferLessThan180 (boolean): is this transfer angle less than 180 degrees?
#TOF (int or float): desired time of flight (HOURS)
#info (list): relevant info, such as gravitational parameter
#isVerbose (boolean): output verbose output or not
#printResults (boolean): print results or not

def Lambert(state1, state2, transferLessThan180, TOF, info, isVerbose, printResults):
    
    R1 = state1[0]
    V1 = state1[1]
    R2 = state2[0]
    V2 = state2[1]
    
    r1 = np.linalg.norm(R1)
    v1 = np.linalg.norm(V1)
    r2 = np.linalg.norm(R2)
    v2 = np.linalg.norm(V2)
    
    mu = info[0]
    
    if transferLessThan180:
        deltaTheta = np.arccos(np.dot(R1,R2)/(r1*r2))
    else:
        deltaTheta = 2*np.pi - np.arccos(np.dot(R1,R2)/(r1*r2))
    
    if isVerbose:
        print("Delta theta = ", np.degrees(deltaTheta), " degrees.")
        
    
    c = np.sqrt(r1**2 + r2**2 - 2*r1*r2*np.cos(deltaTheta))
    s = 0.5*(r1+r2+c)
    
    if isVerbose:
        print('c = ', c)
        print('s = ', s)
    
    if transferLessThan180:
        TOFParabolic = 1/3*np.sqrt(2/mu)*(s**(3/2)-(s-c)**(3/2))/3600 #hours
    else:
        TOFParabolic = 1/3*np.sqrt(2/mu)*(s**(3/2)+(s-c)**(3/2))/3600 #hours
    if isVerbose:
        print("Parabolic TOF = ", TOFParabolic, " hours")
        print("Your TOF = ", TOF, " hours")
    
    isEllipse = False
    isHyperbola = False
    isParabola = False
    
    if TOF > TOFParabolic:
        isEllipse = True
        if isVerbose:
            print("Elliptical Transfer.")
    elif TOF < TOFParabolic:
        isHyperbola = True
        if isVerbose:
            print("Hyperbolic Transfer.")
    else:
        isParabola = True
        if isVerbose:
            print("Parabolic Transfer.")
    
    #Minimum energy arc parameters
    am = s/2
    nm = np.sqrt(mu/(am**3))
    alpham = np.pi
    betam0 = 2*np.arcsin(np.sqrt((s-c)/s))
    
    if transferLessThan180:
        betam = betam0
    else:
        betam = -betam0
    
    TOFm = (1/nm)*(alpham - betam - (np.sin(alpham) - np.sin(betam)))/3600 #hours
    if isVerbose:
        print("Minimum energy ellipse am = ", am, ' km.')
        print("Minimum energy ellipse nm = ", nm)
        print("Minimum energy ellipse betam0 = ", betam0)
        print("Minimum energy ellipse TOF = ", TOFm, ' hours.')
    
    #short or long way
    
    shortWay = False
    longWay = False
    
    if TOF < TOFm:
        shortWay = True
    elif TOF > TOFm:
        longWay = True
    elif TOF == TOFm:
        print("Time of flight indicates minimum energy ellipse transfer.")
    
    if isVerbose:
        print("Short way: ", shortWay)
        print("Long way: ", longWay)
    
    solverVerbose = False
    
    infoLambert = [mu, s, c, TOF, transferLessThan180, shortWay, solverVerbose]
    
    #Solver params
    deltaa1 = am/100
    deltaa2 = am/75
    tolerance = 1e-7
    numberIterations = 1000
    
    a = solve(TOFLambertSolve, infoLambert, am+deltaa1, am+deltaa1+(np.random.rand()*deltaa1), tolerance, numberIterations, solverVerbose)
    #a = opt.fsolve(TOFLambertSolve, am+deltaa1, args=infoLambert)
    
    if isVerbose:
        print("In Lambert: Final a = ", a, " km")
        print("In Lambert: Final TOFi is: ", TOFLambert(a, infoLambert), ' hours')
    
    #Choose alpha and beta
    if transferLessThan180:
        beta = 2*(np.arcsin(np.sqrt((s - c)/(2*a))))
    else:
        beta = -2*(np.arcsin(np.sqrt((s - c)/(2*a))))
        
    if shortWay:
        alpha = 2*np.arcsin(np.sqrt(s/(2*a)))
    elif longWay:
        alpha = 2*np.pi-2*np.arcsin(np.sqrt(s/(2*a)))
      
    #Choose which ellipse
    p1 = ((4*a*(s-r1)*(s-r2))/(c**2))*(np.sin((alpha + beta)/2))**2
    p2 = ((4*a*(s-r1)*(s-r2))/(c**2))*(np.sin((alpha - beta)/2))**2
    pVec = [p1, p2]
    eVec = np.sqrt(1-(pVec/a))
    
    if isVerbose:
        print("Possible p values: ", pVec, " km")
        print("Possible e values: ", eVec)
        
    if transferLessThan180:
        if shortWay:
            eMag = min(eVec)
            p = max(pVec)
        elif longWay:
            eMag = max(eVec)
            p = min(pVec)
    else:
        if shortWay:
            eMag = max(eVec)
            p = min(pVec)
        elif longWay:
            eMag = min(eVec)
            p = max(pVec)
    
    if isVerbose:
        print("Selected p value = ", p, " km")
        print("Selected e value = ", eMag)
        
    h = np.sqrt(mu*p)
    E = mu**2*(eMag**2 - 1)/(2*h**2)
    
    #Find true anomalies
    theta1Vec = [np.arccos(1/eMag*(p/r1 - 1)), 2*np.pi-np.arccos(1/eMag*(p/r1 - 1))]
    theta2Vec = [np.arccos(1/eMag*(p/r2 - 1)), 2*np.pi-np.arccos(1/eMag*(p/r2 - 1))]
    
    combos = [theta2Vec[0] - theta1Vec[0], 
             theta2Vec[1] - theta1Vec[0],
             theta2Vec[0] - theta1Vec[1],
             theta2Vec[1] - theta1Vec[1]]
    
    for i in range(0, len(combos)):
        if combos[i] < 0:
            combos[i] = combos[i] + 2*np.pi
    
    if isVerbose:
        print("Possible theta1 values: ", np.degrees(theta1Vec))
        print("Possible theta2 values: ", np.degrees(theta2Vec))
        print("Combos:")
        print(np.degrees(theta2Vec[0] - theta1Vec[0]))
        print(np.degrees(theta2Vec[1] - theta1Vec[0]))
        print(np.degrees(theta2Vec[0] - theta1Vec[1]))
        print(np.degrees(theta2Vec[1] - theta1Vec[1]))
    
    if(np.abs(combos[0] - deltaTheta) < 0.001):
        theta1 = theta1Vec[0]
        theta2 = theta2Vec[0]
        combosIndex = 0
    elif(np.abs(combos[1] - deltaTheta) < 0.001):
        theta1 = theta1Vec[0]
        theta2 = theta2Vec[1]
        combosIndex = 1
    elif(np.abs(combos[2] - deltaTheta) < 0.001):
        theta1 = theta1Vec[1]
        theta2 = theta2Vec[0]
        combosIndex = 2
    elif(np.abs(combos[3] - deltaTheta) < 0.001):
        theta1 = theta1Vec[1]
        theta2 = theta2Vec[1]
        combosIndex = 3
    
    if isVerbose:
        print("Theta1 = ", np.degrees(theta1))
        print("Theta2 = ", np.degrees(theta2))
        print("Difference = ", np.degrees(combos[combosIndex]))
        print("Delta theta = ", np.degrees(deltaTheta))
        
    #sigma = np.dot(R1,V1)/np.sqrt(mu)
    
    f = 1-((r2/p)*(1-np.cos(deltaTheta)))
    g = r1*r2/np.sqrt(mu*p)*(np.sin(deltaTheta))

    fdot = np.sqrt(mu/p)*np.tan(deltaTheta/2)*(((1-np.cos(deltaTheta))/p)-(1/r2)-(1/r1))
    #fdot = np.sqrt(mu)/(r1*p)*(sigma*(1-np.cos(deltaTheta))-np.sqrt(p)*np.sin(deltaTheta))
    gdot = 1-((r1/p)*(1-np.cos(deltaTheta)))

    V1TransInertial = (R2 - np.multiply(f, R1))/g
    V2TransInertial = np.multiply(fdot, R1) + np.multiply(gdot, V1TransInertial)
    
    #Delta vs
    deltav1 = V1TransInertial - V1
    deltav2 = V2 - V2TransInertial
    deltav1Mag = np.linalg.norm(deltav1)
    deltav2Mag = np.linalg.norm(deltav2)
    totalDeltav = deltav1Mag + deltav2Mag
    
    if printResults:
        print("In Lambert:")
        print("\nTransfer solved.\n")
        print("Initial State (inertial frame):\n")
        print("\tInitial Position = ", R1, "km")
        print("\tInitial Velocity = ", V1, "km")
        print("\tFinal Position = ", R2, "km")
        print("\tFinal Velocity = ", V2, "km\n")
        print("Transfer:\n")
        print("\tSemi-Major Axis a = ", a, " km")
        print("\tEccentricity e = ", eMag)
        print("\tOrbital Energy E = ", E, "m^2/s^2")
        print("\tTrue Anomaly at Transfer Start (theta1) = ", np.degrees(theta1), "degrees")
        print("\tTrue Anomaly at Transfer Start (theta1) = ", np.degrees(theta2), "degrees")
        print("\tInitial Transfer Arc Velocity (inertial frame) = ", V1TransInertial, "km/s")
        print("\tFinal Transfer Arc Velocity (inertial frame) = ", V2TransInertial, "km/s")
        print("\tDelta v_1 = ", deltav1Mag, "km/s")
        print("\tDelta v_2 = ", deltav2Mag, "km/s")
        print("\tTotal delta v = ", totalDeltav, "km/s")
        
    return([a, eMag, E, am, h, p, theta1, theta2, V1TransInertial, V2TransInertial, deltav1Mag, deltav2Mag, totalDeltav])


## Problem 2

In [184]:
#Mars centered inertial frame

Jdate1 = 32503 #UTC
position11 = [-2609, 6305, 8678] #km
velocity11 = [-1.78, -0.272, 0.560] #km/s

Jdate2 = 32503.25 #UTC
position21 = [-14943, -8977, -5773] #km
velocity21 = [0.730, -0.350, -1.025] #km/s

### Part (a)

In [185]:
TOF1 = (Jdate2-Jdate1)*24 #days to hours
transferBelow180 = True

s11 = [position11, velocity11]
s21 = [position21, velocity21]

muMars = 4.305e4 #km^3/s^2

info1 = [muMars]

results_1a = Lambert(s11, s21, transferBelow180, TOF1, info1, True, True)

Delta theta =  109.512919959  degrees.
c =  24382.3395309
s =  26892.4551252
Parabolic TOF =  2.70387002231  hours
Your TOF =  6.0  hours
Elliptical Transfer.
Minimum energy ellipse am =  13446.2275626  km.
Minimum energy ellipse nm =  0.000133071757431
Minimum energy ellipse betam0 =  0.62095715013
Minimum energy ellipse TOF =  6.47614127203  hours.
Short way:  True
Long way:  False

In solve: Successful Solve.

In Lambert: Final a =  13496.6878278  km
In Lambert: Final TOFi is:  6.0  hours
Possible p values:  [11535.033822858264, 10664.484734446038]  km
Possible e values:  [ 0.38123925  0.45808766]
Selected p value =  11535.0338229  km
Selected e value =  0.381239247059
Possible theta1 values:  [  83.23634807  276.76365193]
Possible theta2 values:  [ 167.25073197  192.74926803]
Combos:
84.0143839041
109.512919959
-109.512919959
-84.0143839041
Theta1 =  83.2363480684
Theta2 =  192.749268027
Difference =  109.512919959
Delta theta =  109.512919959
In Lambert:

Transfer solved.

Initial

### Part (b)

In [186]:
print("Lower bound for transfer arc semi-major axis = ", results_1a[3], " km")

Lower bound for transfer arc semi-major axis =  13446.2275626  km


### Part (c)

In [187]:
print("True anomaly at transfer start = ", np.degrees(results_1a[6]), "degrees")
print("True anomaly at transfer end = ", np.degrees(results_1a[7]), "degrees")
print("Initial transfer velocity vector = ", results_1a[8], "km/s")
print("Final transfer velocity vector = ", results_1a[9], "km/s")
print("Delta v_1 magnitude = ", results_1a[10], "km/s")
print("Delta v_2 magnitude = ", results_1a[11], "km/s")
print("Total delta v = ", results_1a[12], "km/s")

True anomaly at transfer start =  83.2363480684 degrees
True anomaly at transfer end =  192.749268027 degrees
Initial transfer velocity vector =  [-2.08463819 -0.22066783  0.46398186] km/s
Final transfer velocity vector =  [ 0.78647957 -0.44563602 -0.82577896] km/s
Delta v_1 magnitude =  0.323510286942 km/s
Delta v_2 magnitude =  0.228090361417 km/s
Total delta v =  0.551600648359 km/s


### Part (d)

In [207]:
elements_2d = orbitalElements(position11, results_1a[8], muMars, True)
rp2d = elements_2d[0]*(1 - elements_2d[1])
print(rp2d)

Semi major axis a =  13496.6878278  km
Eccentricity e =  0.381239247059
Inclination angle i =  52.0004066267  degrees
Right ascension of ascending node =  16.0003446687  degrees
Argument of periapsis =  10.763289002  degrees
True anomaly =  83.2363480684  degrees
8351.22072257


### Part (e)

In [189]:
elements_2e_i = orbitalElements(position11, velocity11, muMars, True)
print()
elements_2e_f = orbitalElements(position21, velocity21, muMars, True)

Semi major axis a =  10145.2208466  km
Eccentricity e =  0.382971152944
Inclination angle i =  51.8227072662  degrees
Right ascension of ascending node =  22.8422073413  degrees
Argument of periapsis =  325.481500775  degrees
True anomaly =  124.294433915  degrees

Semi major axis a =  14433.2367819  km
Eccentricity e =  0.282210140148
Inclination angle i =  60.4790968714  degrees
Right ascension of ascending node =  20.186862374  degrees
Argument of periapsis =  9.74687198651  degrees
True anomaly =  191.432059027  degrees


### Part (f)

In [190]:
results_1f = Lambert(s11, s21, False, TOF1, info1, True, True)

Delta theta =  250.487080041  degrees.
c =  24382.3395309
s =  26892.4551252
Parabolic TOF =  2.86260595305  hours
Your TOF =  6.0  hours
Elliptical Transfer.
Minimum energy ellipse am =  13446.2275626  km.
Minimum energy ellipse nm =  0.000133071757431
Minimum energy ellipse betam0 =  0.62095715013
Minimum energy ellipse TOF =  6.6395587509  hours.
Short way:  True
Long way:  False

In solve: Successful Solve.

In Lambert: Final a =  13542.2887016  km
In Lambert: Final TOFi is:  6.0  hours
Possible p values:  [10507.760480927829, 11707.079956615902]  km
Possible e values:  [ 0.47336872  0.36812618]
Selected p value =  10507.7604809  km
Selected e value =  0.473368719752
Possible theta1 values:  [  95.83875087  264.16124913]
Possible theta2 values:  [ 154.64832917  205.35167083]
Combos:
58.809578309
109.512919959
-109.512919959
-58.809578309
Theta1 =  264.161249134
Theta2 =  154.648329175
Difference =  250.487080041
Delta theta =  250.487080041
In Lambert:

Transfer solved.

Initial St

### Part (i)

In [215]:
r1trans1 = np.linalg.norm(position11)
r2trans1 = np.linalg.norm(position21)
r1trans2 = np.linalg.norm(position11)
r2trans2 = np.linalg.norm(position21)

print("First transfer initial position mag = ", r1trans1, "km")
print("First transfer final position mag = ", r2trans1, "km")
print("Second transfer initial position mag = ", r1trans2, "km")
print("Second transfer final position mag = ", r2trans2, "km")

First transfer initial position mag =  11039.3654709 km
First transfer final position mag =  18363.2052485 km
Second transfer initial position mag =  11039.3654709 km
Second transfer final position mag =  18363.2052485 km


## Problem 3

In [191]:
position12 = [8990, 2219, 59.40] #km
velocity12 = [-0.500, 2.105, -0.039] #km/s

position22 = [19888, -12412, 748] #km
velocity22 = [0.714, 1.147, 0.030] #km/s

s12 = [position12, velocity12]
s22 = [position22, velocity22]

info2 = [muMars]

### Part (a)

In [199]:
#lessthan180
TOF2 = 15 #hours
ty
results_3a = Lambert(s12, s22, True, TOF2, info2, True, True)

Delta theta =  45.8463503784  degrees.
c =  18256.6901425
s =  25485.9808629
Parabolic TOF =  2.17984800263  hours
Your TOF =  15  hours
Elliptical Transfer.
Minimum energy ellipse am =  12742.9904315  km.
Minimum energy ellipse nm =  0.000144237950365
Minimum energy ellipse betam0 =  1.12332754682
Minimum energy ellipse TOF =  5.62305842034  hours.
Short way:  False
Long way:  True

In solve: Successful Solve.

In Lambert: Final a =  17231.3218913  km
In Lambert: Final TOFi is:  15.0  hours
Possible p values:  [1919.0975153929955, 6789.473061181764]  km
Possible e values:  [ 0.94267034  0.77844763]
Selected p value =  1919.09751539  km
Selected e value =  0.942670343801
Possible theta1 values:  [ 147.24230456  212.75769544]
Possible theta2 values:  [ 166.91134506  193.08865494]
Combos:
19.669040501
45.8463503784
-45.8463503784
-19.669040501
Theta1 =  147.24230456
Theta2 =  193.088654939
Difference =  45.8463503784
Delta theta =  45.8463503784
In Lambert:

Transfer solved.

Initial Sta

### Part (b)

In [193]:
print("True anomaly at transfer start = ", np.degrees(results_3a[6]), "degrees")
print("True anomaly at transfer end = ", np.degrees(results_3a[7]), "degrees")
print("Initial transfer velocity vector = ", results_3a[8], "km/s")
print("Final transfer velocity vector = ", results_3a[9], "km/s")
print("Delta v_1 magnitude = ", results_3a[10], "km/s")
print("Delta v_2 magnitude = ", results_3a[11], "km/s")
print("Total delta v = ", results_3a[12], "km/s")

True anomaly at transfer start =  147.24230456 degrees
True anomaly at transfer end =  193.088654939 degrees
Initial transfer velocity vector =  [ 2.58020191 -0.37342615  0.05301294] km/s
Final transfer velocity vector =  [-1.06266097  0.20651543 -0.02371018] km/s
Delta v_1 magnitude =  3.95458042948 km/s
Delta v_2 magnitude =  2.01095007742 km/s
Total delta v =  5.96553050689 km/s


### Part (c) 

(Do you think that the total ∆𝑣 required for this transfer would be feasible for a crewed exploration of the Martian system?)

No.

### Part (d)

In [194]:
#Use f and g functions to calculate Deimos state at t3

dt = 12 #hours
positionDeimos = position22
velocityDeimos = velocity22

#RD = [5024, -751, -2286]
#VD = [-0.436, 0.135, 2.2]

elements_3d = orbitalElements(positionDeimos, velocityDeimos, muMars, False)
#return([a, e, i, RAAN, aop, theta, h, p])

#def fgE(Ri, Vi, rf, dt, a, mu, deltaE):
#def fgTheta(Ri, Vi, rf, mu, p, deltaTheta):

E1 = theta2E(elements_3d[5], elements_3d[1])
n3d = np.sqrt(muMars/(elements_3d[0]**3))

info_3d = [n3d, elements_3d[1], dt, E1]
#def solve(func, info, x0, x1, err, Nmax, isVerbose):
E2 = solve(Kepler, info_3d, 1, 0, 1e-7, 10000, False)
thetaD2 = E2theta(E2, elements_3d[1])

rD2 = elements_3d[7]/(1+elements_3d[1]*np.cos(thetaD2))

deltaE = E2-E1
deltaThetaDeimos = thetaD2 - elements_3d[5]

t3State = fgTheta(positionDeimos, velocityDeimos, rD2, muMars, elements_3d[7], deltaThetaDeimos)

elements_3dFinal = orbitalElements(t3State[0], t3State[1], muMars, False)

print("Position Vector of Deimos at t3 = ", t3State[0], "km")
print("Velocity Vector of Deimos at t3 = ", t3State[1], "km")


In solve: Successful Solve.

Position Vector of Deimos at t3 =  [ -8639.88673164  21572.14599074   -292.98956806] km
Velocity Vector of Deimos at t3 =  [-1.263768   -0.51059885 -0.05009774] km


### Part (e)

In [201]:
results_3e = Lambert(s12, t3State, True, 27, info2, True, True)

Delta theta =  97.9653706772  degrees.
c =  26181.6986522
s =  29340.777699
Parabolic TOF =  3.05978736487  hours
Your TOF =  27  hours
Elliptical Transfer.
Minimum energy ellipse am =  14670.3888495  km.
Minimum energy ellipse nm =  0.000116768080196
Minimum energy ellipse betam0 =  0.668644170701
Minimum energy ellipse TOF =  7.35758333459  hours.
Short way:  False
Long way:  True

In solve: Successful Solve.

In Lambert: Final a =  23961.1889063  km
In Lambert: Final TOFi is:  27.0  hours
Possible p values:  [6092.1198758112423, 14376.319205860453]  km
Possible e values:  [ 0.86356848  0.63246854]
Selected p value =  6092.11987581  km
Selected e value =  0.863568478845
Possible theta1 values:  [ 113.33778757  246.66221243]
Possible theta2 values:  [ 148.69684175  211.30315825]
Combos:
35.3590541747
97.9653706772
-97.9653706772
-35.3590541747
Theta1 =  113.337787574
Theta2 =  211.303158251
Difference =  97.9653706772
Delta theta =  97.9653706772
In Lambert:

Transfer solved.

Initial

### Part (f)

In [196]:
print("True anomaly at transfer start = ", np.degrees(results_3e[6]), "degrees")
print("True anomaly at transfer end = ", np.degrees(results_3e[7]), "degrees")
print("Initial transfer velocity vector = ", results_3e[8], "km/s")
print("Final transfer velocity vector = ", results_3e[9], "km/s")
print("Delta v_1 magnitude = ", results_3e[10], "km/s")
print("Delta v_2 magnitude = ", results_3e[11], "km/s")
print("Total delta v = ", results_3e[12], "km/s")

True anomaly at transfer start =  113.337787574 degrees
True anomaly at transfer end =  211.303158251 degrees
Initial transfer velocity vector =  [ 1.62740265  2.20293225 -0.00717264] km/s
Final transfer velocity vector =  [-0.20345003 -1.3662564   0.01175259] km/s
Delta v_1 magnitude =  2.1298933625 km/s
Delta v_2 magnitude =  1.3639096344 km/s
Total delta v =  3.4938029969 km/s


In [205]:
#TEST

#greaterthan180
pa = [-654, 13605, 1997]
va = [-5.53, 0.849, 0.6830]

pb = [7284, -19341, -3264]
vb = [3.07, 2.63, 0.444]

infotest = [3.986004415e5]

sa = [pa, va]
sb = [pb, vb]

TOFt = 5 #hrs

t1 = time.clock()
resultstest = Lambert(sa, sb, False, TOFt, infotest, False, True)
t2 = time.clock()
print("Runtime = ", t2-t1, "seconds")


In solve: Successful Solve.

In Lambert:

Transfer solved.

Initial State (inertial frame):

	Initial Position =  [-654, 13605, 1997] km
	Initial Velocity =  [-5.53, 0.849, 0.683] km
	Final Position =  [7284, -19341, -3264] km
	Final Velocity =  [3.07, 2.63, 0.444] km

Transfer:

	Semi-Major Axis a =  19001.2103585  km
	Eccentricity e =  0.310047201662
	Orbital Energy E =  -10.4888171327 m^2/s^2
	True Anomaly at Transfer Start (theta1) =  37.009671933 degrees
	True Anomaly at Transfer Start (theta1) =  234.700206695 degrees
	Initial Transfer Arc Velocity (inertial frame) =  [-6.03306023  0.54895508  0.48237226] km/s
	Final Transfer Arc Velocity (inertial frame) =  [ 3.27345533  2.52730206  0.14387584] km/s
	Delta v_1 =  0.619151058392 km/s
	Delta v_2 =  0.376849372654 km/s
	Total delta v =  0.996000431046 km/s
Runtime =  0.007083000000001505 seconds


In [212]:
phobosElements = orbitalElements(position12, velocity12, muMars, True)
print("\nrp phobos = ", phobosElements[0]*(1 - phobosElements[1]))
print("ra phobos = ", phobosElements[0]*(1 + phobosElements[1]))
print()
deimosElements = orbitalElements(position22, velocity22, muMars, True)
print("\nrp deimos = ", deimosElements[0]*(1 - deimosElements[1]))
print("ra deimos = ", deimosElements[0]*(1 + deimosElements[1]))

Semi major axis a =  9327.24512977  km
Eccentricity e =  0.0112738622862
Inclination angle i =  1.0991959235  degrees
Right ascension of ascending node =  -146.60288179  degrees
Argument of periapsis =  109.720627358  degrees
True anomaly =  50.7440942271  degrees

rp phobos =  9222.09105266
ra phobos =  9432.39920687

Semi major axis a =  23339.5129544  km
Eccentricity e =  0.00497967976275
Inclination angle i =  2.22731929287  degrees
Right ascension of ascending node =  -87.0887181447  degrees
Argument of periapsis =  230.044356419  degrees
True anomaly =  185.096595734  degrees

rp deimos =  23223.2896541
ra deimos =  23455.7362548
